# 机器学习第一次实验

## 任务一：理解对数几率回归（Logistic Regression）算法原理

### 对数几率回归的核心

- **概率模型**：对数几率回归的目标是预测给定输入数据点属于某一类的概率。它通过将数据特征的线性组合输入到逻辑函数（通常是Sigmoid函数）中，后将输出转换为概率值。

- **Sigmoid函数**：
  $$
  \sigma(z) = \frac{1}{1 + e^{-z}}
  $$
  其中，$z$是特征与权重的线性组合。Sigmoid函数的输出范围是$(0,1)$，非常适合表示概率。对于给定的输入特征$x$，$z$可以表示为$z = w^T x + b$，其中$w$是权重向量，$b$是偏置项。

- **决策边界**：对数几率回归模型通过预测概率来做出分类决策。通常情况下，如果预测概率大于或等于$0.5$，模型会将数据点分类为正类（标签为$1$）；否则，分类为负类（标签为$0$）。

### 损失函数与参数优化

- **损失函数（对数损失）**：在对数几率回归中，使用对数损失函数来度量模型预测值与实际标签之间的差异。对于单个数据点，损失函数定义为：
  $$
  L(y, \hat{y}) = -[y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
  $$
  其中，$y$是真实标签，$\hat{y}$是预测概率。对于整个数据集，损失是所有单个损失的平均值。

- **参数优化（梯度下降）**：对数几率回归使用梯度下降算法来优化模型参数（权重$w$和偏置$b$），以最小化损失函数。梯度下降是一种迭代优化算法，通过不断更新参数来逐步减少损失函数的值。

## 任务二：编程实现对数几率回归算法

以下是对数几率回归算法设计

In [1]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        
        # 梯度下降
        for _ in range(self.num_iterations):
            # 计算模型预测值
            model = np.dot(X, self.weights) + self.bias
            predictions = self._sigmoid(model)
            
            # 计算梯度
            dw = (1 / num_samples) * np.dot(X.T, (predictions - y))
            db = (1 / num_samples) * np.sum(predictions - y)
            
            # 更新参数
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        predictions = self._sigmoid(model)
        prediction_class = [1 if i > 0.5 else 0 for i in predictions]
        return prediction_class

## 任务三：将算法应用于具体数据集来完成分类问题

In [2]:
# 首先研究鸢尾花数据集分类问题

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# 加载数据
iris_data = pd.read_csv('data/iris.csv')
print(iris_data.head())

   Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
1           5.1          3.5           1.4          0.2  setosa
2           4.9          3.0           1.4          0.2  setosa
3           4.7          3.2           1.3          0.2  setosa
4           4.6          3.1           1.5          0.2  setosa
5           5.0          3.6           1.4          0.2  setosa


In [3]:
import warnings
warnings.filterwarnings('ignore')

# 转化为二分类
iris_data = iris_data[iris_data['Species'] != 'virginica']
iris_data['Species'] = iris_data['Species'].map({'setosa': 0, 'versicolor': 1})
iris_data = shuffle(iris_data, random_state=42)

# 划分数据集
X = iris_data.drop('Species', axis=1)
y = iris_data['Species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(iris_data.head())

(80, 4) (20, 4) (80,) (20,)
    Sepal.Length  Sepal.Width  Petal.Length  Petal.Width  Species
84           6.0          2.7           5.1          1.6        1
54           5.5          2.3           4.0          1.3        1
71           5.9          3.2           4.8          1.8        1
46           4.8          3.0           1.4          0.3        0
45           5.1          3.8           1.9          0.4        0


In [4]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 训练模型
model = LogisticRegression(learning_rate=0.1, num_iterations=1000)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# 计算准确率
accuracy = np.mean(predictions == y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


### 结果分析

将对数几率回归算法应用到鸢尾花数据集上，通过运行代码得到预测准确率高达 $100\%$，这一方面显示出该算法在该数据集上预测性能极好，另一方面也反映出有一定程度地过拟合，这可能是由于该数据集较小且特征之前区分度过大造成的

## 下面是对一个新的数据集进行研究了

In [6]:
# 然后我们继续研究西瓜数据集

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

watermelon_data = pd.read_csv('data/watermelon3_0_ch.csv')
print(watermelon_data.head())

   编号  色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率 好瓜
0   1  青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460  是
1   2  乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376  是
2   3  乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264  是
3   4  青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318  是
4   5  浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215  是


In [7]:
watermelon_data = watermelon_data.drop('编号', axis=1)
watermelon_data['好瓜'] = watermelon_data['好瓜'].map({'是': 1, '否': 0})
watermelon_data['色泽'] = watermelon_data['色泽'].map({'青绿': 0, '乌黑': 1, '浅白': 2})
watermelon_data['根蒂'] = watermelon_data['根蒂'].map({'蜷缩': 0, '稍蜷': 1, '硬挺': 2})
watermelon_data['敲声'] = watermelon_data['敲声'].map({'浊响': 0, '沉闷': 1, '清脆': 2})
watermelon_data['纹理'] = watermelon_data['纹理'].map({'清晰': 0, '稍糊': 1, '模糊': 2})
watermelon_data['脐部'] = watermelon_data['脐部'].map({'凹陷': 0, '稍凹': 1, '平坦': 2})
watermelon_data['触感'] = watermelon_data['触感'].map({'硬滑': 0, '软粘': 1})
watermelon_data = shuffle(watermelon_data, random_state=42)

X = watermelon_data.drop('好瓜', axis=1)
y = watermelon_data['好瓜']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(watermelon_data.head())

(13, 8) (4, 8) (13,) (4,)
    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率  好瓜
0    0   0   0   0   0   0  0.697  0.460   1
1    1   0   1   0   0   0  0.774  0.376   1
5    0   1   0   0   1   1  0.403  0.237   1
15   2   0   0   2   2   0  0.593  0.042   0
11   2   0   0   2   2   1  0.343  0.099   0


In [8]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 训练模型
model = LogisticRegression(learning_rate=0.1, num_iterations=1000)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# 计算准确率
accuracy = np.mean(predictions == y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.75


### 结果分析

将对数几率回归算法应用到西瓜数据集上，通过运行代码得出预测准确率为 $75\%$，这表明该模型预测在该数据集上性能良好且未表现出过拟合现象。